<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>

<h1> Práctico I - Estadística </h1>
<h3> Análisis y Visualización de Datos - 2019 </h3>

Durante este práctico vamos a trabajar sobre el dataset [Human Freedom Index 2018](https://www.cato.org/human-freedom-index-new) de el instituto Cato. Este índice mide en detalle lo que entendemos como libertad, utilizando 79 indicadores de libertad personal y económica en distintos aspectos, hasta obtener un hermoso numerito del 1 al 10. Usaremos una [versión ya limpia del dataset](https://www.kaggle.com/gsutters/the-human-freedom-index/home) que pueden descargar desde Kaggle.

Las variables más importantes sobre las que trabaja el dataset son:

* Rule of Law
* Security and Safety
* Movement
* Religion
* Association, Assembly, and Civil Society
* Expression and Information
* Identity and Relationships
* Size of Government
* Legal System and Property Rights
* Access to Sound Money
* Freedom to Trade Internationally
* Regulation of Credit, Labor, and Business

Nosotros centrarermos nuestro análisis en variables relacionadas a *Identity and Relationships* en paises de Latinoamérica, y los compararemos con las estadísticas globales. La pregunta a responder es simple: **¿Qué niveles de libertad se viven en Latinoamérica, especificamente en cuanto libertades de indentidad?**. Sin embargo, para hacer un análisis de los datos tenemos que platear también estas sub preguntas:

1. ¿Qué significa tener un puntaje de 4.5? Hay que poner los puntajes de la región en contexto con los datos del resto del mundo.
2. ¿Cuál es la tendencia a lo largo de los años? ¿Estamos mejorando, empeorando?
3. En este estudio, la libertad se mide con dos estimadores principales: *hf_score* que hace referencia a Human Freedom, y *ef_score* que hace referencia a Economic Freedom. Estos dos estimadores, ¿se relacionan de la misma manera con la libertad de identidad?

Inicialmente, en toda exploración de datos tenemos muy poca información a priori sobre el significado de los datos y tenemos que empezar por comprenderlos. Les proponemos los siguientes ejercicios como guía para comenzar esta exploración.

In [3]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
from scipy import stats

In [4]:
seaborn.__version__

'0.9.0'

In [5]:
dataset = pandas.read_csv('hfi_cc_2018.csv')
dataset.shape

(1458, 123)

In [6]:
dataset.columns  # Way too many columns!

Index(['year', 'ISO_code', 'countries', 'region', 'pf_rol_procedural',
       'pf_rol_civil', 'pf_rol_criminal', 'pf_rol', 'pf_ss_homicide',
       'pf_ss_disappearances_disap',
       ...
       'ef_regulation_business_bribes', 'ef_regulation_business_licensing',
       'ef_regulation_business_compliance', 'ef_regulation_business',
       'ef_regulation', 'ef_score', 'ef_rank', 'hf_score', 'hf_rank',
       'hf_quartile'],
      dtype='object', length=123)

Por suerte las columnas tienen un prefijo que nos ayuda a identificar a qué sección pertenecen. Nos quedamos sólo con las que comienzan con *pf_indentity*, junto con otras columnas más generales

In [7]:
important_cols = ['year', 'ISO_code', 'countries', 'region']

important_cols += [col for col in dataset.columns if 'pf_identity' in col]

important_cols += [
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]

# Imprime dataset con las columnas requeridas
#dataset[important_cols]

## 1. Estadísticos descriptivos

  1. Para comenzar con un pantallazo de los datos, calcular el rango de las variables.
  2. Obtener media, mediana y desviación estándar de las variables *pf_identity* y *hf_score* en el mundo y compararla con la de Latinoamérica y el caribe. ¿Tiene sentido calcular la moda? 
  3. ¿Son todos los valores de *pf_identity*  y *hf_score* directamente comparables? ¿Qué otra variable podría influenciarlos?
  4. ¿Cómo pueden sanearse los valores faltantes?
  5. ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? ¿Los outliers, son globales o por grupo? ¿Los eliminaría del conjunto de datos?
  
  

### 0   Variables Globales

In [33]:
dataset_latin_america = dataset[dataset.region == "Latin America & the Caribbean"]

### 1.1   Rango de Variables

#### 1.1.1 Definición de los parametros a examinar

In [34]:
parametros = []
parametros += [col for col in dataset.columns if 'pf_identity' in col]
parametros += [
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]

#### 1.1.2 Minimo y Máximo de los parametros

In [35]:
datos = [(x, dataset[x].min(), dataset[x].max()) for x in dataset[parametros]]
df    = pandas.DataFrame(data=datos, columns=['Variable','Mínimo','Máximo'])
display(df)

,Variable,Mínimo,Máximo
0,pf_identity_legal,0.000000,10.000000
1,pf_identity_parental_marriage,0.000000,10.000000
2,pf_identity_parental_divorce,0.000000,10.000000
3,pf_identity_parental,0.000000,10.000000
4,pf_identity_sex_male,0.000000,10.000000
5,pf_identity_sex_female,0.000000,10.000000
6,pf_identity_sex,0.000000,10.000000
7,pf_identity_divorce,0.000000,10.000000
8,pf_identity,0.000000,10.000000
9,ef_score,2.880000,9.190000


#### 1.1.3 Analisis de los parametros a examinar

In [36]:
for x in dataset[parametros] : 
    max = dataset[x].max()
    min = dataset[x].min()
    print(x + "\n")
    print(x + ": Minimo = " + str(min))
    print(x + ": Maximo = " + str(max))
    print("Datos : " + str(dataset[x].unique()) + "\n" + "--------------------------" + "\n")

pf_identity_legal

pf_identity_legal: Minimo = 0.0
pf_identity_legal: Maximo = 10.0
Datos : [ 0. nan 10.  7.]
--------------------------

pf_identity_parental_marriage

pf_identity_parental_marriage: Minimo = 0.0
pf_identity_parental_marriage: Maximo = 10.0
Datos : [10.  0. nan  5.]
--------------------------

pf_identity_parental_divorce

pf_identity_parental_divorce: Minimo = 0.0
pf_identity_parental_divorce: Maximo = 10.0
Datos : [10.  5.  0. nan]
--------------------------

pf_identity_parental

pf_identity_parental: Minimo = 0.0
pf_identity_parental: Maximo = 10.0
Datos : [10.   2.5  0.   5.   7.5  nan]
--------------------------

pf_identity_sex_male

pf_identity_sex_male: Minimo = 0.0
pf_identity_sex_male: Maximo = 10.0
Datos : [10.  0.  5. nan]
--------------------------

pf_identity_sex_female

pf_identity_sex_female: Minimo = 0.0
pf_identity_sex_female: Maximo = 10.0
Datos : [10.  0.  5. nan]
--------------------------

pf_identity_sex

pf_identity_sex: Minimo = 0.0
pf_identi

#### 1.1.4 Rango de variables

<table>
    <tr>
        <th>Variable</th>
        <th>Rango en Notación de conjunto</th>
    </tr>
    <tr>
        <td>pf_identity_legal</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_parental_marriage</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_parental_divorce</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_parental</td>
        <td>{ x pertenece a Q | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_sex_male</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_sex_female</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_sex</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity_divorce</td>
        <td>{ x pertenece a N | 0 <= x <= 10}  Conjunto finito</td>
    </tr>
    <tr>
        <td>pf_identity</td>
        <td>{ x pertenece a R | 0 <= x <= 10}  Conjunto infinito no contable</td>
    </tr>
    <tr>
        <td>ef_score</td>
        <td>{ x pertenece a Q | 2.88 <= x <= 9.19}  Conjunto finito</td>
    </tr>
    <tr>
        <td>ef_rank</td>
        <td>{ x pertenece a N | 1 <= x <= 162}  Conjunto finito</td>
    </tr>
    <tr>
        <td>hf_score</td>
        <td>{ x pertenece a R | 0 <= x <= 10}  Conjunto infinito no contable</td>
    </tr>
    <tr>
        <td>hf_rank</td>
        <td>{ x pertenece a N | 1 <= x <= 162}  Conjunto finito</td>
    </tr>
</table>

### 1.2   Media, mediana y desviación estándar de las variables pf_identity y hf_score en el mundo y compararla con la de Latinoamérica y el caribe

In [41]:
# Datos Mundiales
pf_identity_mundo_media              = dataset['pf_identity'].mean()
pf_identity_mundo_mediana            = dataset['pf_identity'].median()
pf_identity_mundo_desviacion         = dataset['pf_identity'].std()

# Datos para Ámerica Latina
pf_identity_latin_america_media      = dataset_latin_america['pf_identity'].mean()
pf_identity_latin_america_mediana    = dataset_latin_america['pf_identity'].median()
pf_identity_latin_america_desviacion = dataset_latin_america['pf_identity'].std()

# Datos comparativos

datos_comparativos = [['Mundo',pf_identity_mundo_media, pf_identity_mundo_mediana, pf_identity_mundo_desviacion],
                      ['Ámerica Latina',pf_identity_latin_america_media, pf_identity_latin_america_mediana, pf_identity_latin_america_desviacion]
                     ]

cuadro             = pandas.DataFrame(datos_comparativos, columns=['Lugar','Media', 'Mediana', 'Desviación'])
display(cuadro)


,Lugar,Media,Mediana,Desviación
0,Mundo,7.334180,9.166667,3.159473
1,Ámerica Latina,8.765086,10.000000,1.711732


#### Análisis de la comparación

Deacuerdo a estos datos el indice de pf_identity en latinoamerica es más alto que el Mundial, con una desviación menor (que indica la similitud de los paises de Ámerica Latina y el Caribe) 

#### ¿Tiene sentido calcular la moda?

No tiene sentido calcular la moda, dado que los datos no son discretos, son variables continuas

## 2. Agregación de datos

1. Grafiquen la media de la variable *pf_identity* y *hf_score* a través de los años.
2. Realicen los mismos gráficos, pero separando por regiones (Cada variable en un gráfico distinto, sino no se ve nada). ¿La tendencia observada, es la misma que si no dividimos por regiones?
3. Si lo consideran necesario, grafiquen algunos países de Latinoamerica para tratar de explicar la tendencia de la variable *pf_identity* en la región. ¿Cómo seleccionarion los países relevantes a esa tendencia?

Hint: hay un gráfico de seaborn que hace todo por vos!

Sólo por curiosidad, graficar la tendencia de *hf_score* y *ef_score* a través de los años. ¿Tienen alguna hipótesis para este comportamiento?

### Gráfico de la media de pf_identity y hf_score a lo largo del tiempo para la región de Latinoamérica y Caribe.

In [ ]:
#america latina

dataset_imp_cols = dataset[important_cols]
latin_american=dataset_imp_cols[dataset_imp_cols.region == 'Latin America & the Caribbean']

years = list(latin_american.year.unique())
years.reverse()

pf_identity_med = []
hf_score_med = []

for y in years:
    pf_identity_med.append(
        latin_american[latin_american.year == y].pf_identity.mean() 
    )
    hf_score_med.append(
        latin_american[latin_american.year == y].hf_score.mean() 
    )

df1 = pandas.DataFrame(dict(years=years, pf_identity_media=pf_identity_med))
df2 = pandas.DataFrame(dict(years=years, hf_score_media=hf_score_med))


seaborn.lineplot(x='years', y='pf_identity_media', data=df1)
plt.figure()
seaborn.lineplot(x='years', y='hf_score_media', data=df2, color='red')

fig, ax = plt.subplots()
seaborn.regplot(x='years', y='pf_identity_media', data=df1, ax=ax)
ax2 = ax.twinx()
seaborn.regplot(x='years', y='hf_score_media', data=df2, ax=ax2, color='r')

### Grafico de la media de pf_identity y hf_score a lo largo de los años a nivel global, salvo región Latinoamérica y Caribe.

In [ ]:
#resto del mundo menos america latina y caribe

dataset_imp_cols = dataset[important_cols]
latin_american=dataset_imp_cols[dataset_imp_cols.region != 'Latin America & the Caribbean']

years = list(latin_american.year.unique())
years.reverse()

pf_identity_med = []
hf_score_med = []

for y in years:
    pf_identity_med.append(
        latin_american[latin_american.year == y].pf_identity.mean() 
    )
    hf_score_med.append(
        latin_american[latin_american.year == y].hf_score.mean() 
    )

df1 = pandas.DataFrame(dict(years=years, pf_identity_media=pf_identity_med))
df2 = pandas.DataFrame(dict(years=years, hf_score_media=hf_score_med))


seaborn.lineplot(x='years', y='pf_identity_media', data=df1)
plt.figure()
seaborn.lineplot(x='years', y='hf_score_media', data=df2, color='red')

fig, ax = plt.subplots()
seaborn.regplot(x='years', y='pf_identity_media', data=df1, ax=ax)
ax2 = ax.twinx()
seaborn.regplot(x='years', y='hf_score_media', data=df2, ax=ax2, color='r')

## 3. Distribuciones
  1. Graficar en un mismo histograma la distribución de la variable *pf_identity* en global, y en Latinoamérica y el caribe. Repetir para la variable *hf_score*. ¿Visualmente, a qué tipo de distribución corresponde cada variable? ¿Es correcto utilizar todos los registros para esas zonas en estos gráficos?
  2. Realizar una prueba de Kolmogorov-Smirnof para comprobar analíticamente si estas variables responden la distribución propuesta en el ejercicio anterior. Hint: podés usar https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kstest.html, pero hay que tener en cuenta que si la distribución es "norm", entonces va a comparar los datos con una distribución normal con media 0 y desviación estándar 1. Se puede utilizar la distribución sobre todos los datos o sólo sobre Latinoamérica.
  3. Realizar un gráfico QQ de las mismas distribuciones. Se puede utilizar a,bas distribuciones sobre todos los datos o sólo sobre Latinoamérica, pero no cruzadas.
  4. Medir la asimetría (skew) y curtosis de las mismas distribuciones con las que realizó el gráfico anterior. ¿Cómo se relacionan estos estadísticos con la forma del gráfico QQ obtenido previamente? ¿El gráfico QQ provee más información que no esté presente en estos estadísticos?

### 3.1.1 histograma la distribución de la variable pf_identity en global, y en Latinoamérica y el caribe

In [ ]:

plt.figure(figsize=(10,6))
seaborn.distplot(ds['pf_identity'].dropna(), label='Global')
seaborn.distplot(ds[ds.region == "Latin America & the Caribbean"]['pf_identity'].dropna(), label='Latin America')
plt.ylabel('Densidad de probabilidad')
plt.legend()
seaborn.despine()

Analisis <br>
<ul>
    <li>America Latina tiene un indice (de probabilidad) más alto, en libertad de identidad comparado con el mundo</li>
    <li>El tipo de distribución es Normal con cola a la izquiera y no simetrica, para ambos casos (Mundo y America Latina)</li>
</ul>

##### 3.1.2 histograma de distribución de la variable hf_score en global, y en Latinoamérica y el caribe

In [ ]:

plt.figure(figsize=(10,6))
seaborn.distplot(ds['hf_score'].dropna(), label='Global')
seaborn.distplot(ds[ds.region == "Latin America & the Caribbean"]['hf_score'].dropna(), label='Latin America')
plt.ylabel('Densidad de probabilidad')
plt.legend()
seaborn.despine()

Analisis
<ul>
    <li>America Latina tiene un indice (de probabilidad) más alto, en libertades humanas (human freedom score) comparado con el mundo</li>
    <li>El tipo de distribución es Normal no simetrica, para ambos casos (Mundo y America Latina)</li>
</ul>

### 3.2 Realizar una prueba de Kolmogorov-Smirnof para comprobar analíticamente si estas variables responden la distribución propuesta en el ejercicio anterior.  

pero hay que tener en cuenta que si la distribución es "norm", entonces va a comparar los datos con una distribución normal con media 0 y desviación estándar 1. Se puede utilizar la distribución sobre todos los datos o sólo sobre Latinoamérica.

##### pf_ identity en el mundo y America Latina

In [ ]:
stats.kstest(dataset['pf_identity'].dropna(), 'norm')

In [ ]:
stats.kstest(dataset[dataset['region'] == 'Latin America & the Caribbean']['pf_identity'].dropna(), 'norm')

##### hf_score en el mundo y America Latina

In [ ]:
stats.kstest(dataset['hf_score'].dropna(), 'norm')

In [ ]:
stats.kstest(dataset[dataset['region'] == 'Latin America & the Caribbean']['hf_score'].dropna(), 'norm')

Analisis
<ul>
    <li>En ambos casos (pf_identity y hf_score) al optener pvalue = 0, no podemos afirmar que la distribución sea normal </li>
    <li>Dado que el test dice que la distribución no es normal, las distribuciones del punto 3.1.1 y 3.1.2 posiblemente no lo sean </li>
</ul>

### 3.3.1 Gráfico QQ de las distribuciones de pf_identity en el mundo

Primero sacamos los parametros de la media y desviación estándar (de pf_identity en el mundo), para luego hacer una muestra aleatoria que si tenga una distribución normal que vamos a usar como comparador en el gráfico QQ

In [ ]:
from scipy import stats

# Definimos la variable para el dataset que vamos a utilizar
dataset_pf_identity_mundo = dataset['pf_identity'].dropna()

# Estimamos los parámetros muestrales (la media y desviación estándar)
loc, scale = stats.norm.fit(dataset_pf_identity_mundo)

# Creamos una distribución normal
norm_dist = stats.norm(loc, scale)

# Generamos una muestra
sample = norm_dist.rvs(1000)

# La graficamos
seaborn.distplot(sample, label='Distribución normal')
seaborn.distplot(dataset_pf_identity_mundo, label='pf_identity_mundo')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
seaborn.despine()

Ahora podemos armar el gráfico QQ. Usamos dos muestras del mismo tamaño, y calculamos los cuartiles.

In [ ]:
# Creamos 20 puntos percentiles igualmente distribuidos entre 0 y 100.
percs = numpy.linspace(0,100,21)

qn_sample = numpy.percentile(dataset_pf_identity_mundo, percs)
qn_norm_dist = numpy.percentile(norm_dist.rvs(len(dataset_pf_identity_mundo)), percs)

plt.figure(figsize=(12,6))
seaborn.regplot(x=qn_sample, y=qn_norm_dist)

plt.xlabel('Percentiles de la muestra de pf_identity_mundo')
plt.ylabel('Percentiles de la distribución \n normal estimada')
plt.title('Gráfico QQ de la distribución de pf_identity_mundo y una distribución normal')

x = numpy.linspace(numpy.min((qn_sample.min(), qn_norm_dist.min())), numpy.max((qn_sample.max(),qn_norm_dist.max())))
plt.plot(x,x, color="#FF5964", ls="--")

seaborn.despine()

### 3.3.2 Gráfico QQ de las distribuciones de pf_identity en America Latina y el Caribe

Primero sacamos los parametros de la media y desviación estándar, para luego hacer una muestra aleatoria que si tenga una distribución normal que vamos a usar como comparador en el gráfico QQ

In [ ]:
from scipy import stats

# Definimos la variable para el dataset que vamos a utilizar
dataset_pf_identity_latin_america = dataset[dataset['region'] == 'Latin America & the Caribbean']['pf_identity'].dropna()

# Estimamos los parámetros muestrales (la media y desviación estándar)
loc, scale = stats.norm.fit(dataset_pf_identity_latin_america)

# Creamos una distribución normal
norm_dist = stats.norm(loc, scale)

# Generamos una muestra
sample = norm_dist.rvs(1000)

# La graficamos
seaborn.distplot(sample, label='Distribución normal')
seaborn.distplot(dataset_pf_identity_latin_america, label='pf_identity_latin_america')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
seaborn.despine()

Ahora podemos armar el gráfico QQ. Usamos dos muestras del mismo tamaño, y calculamos los cuartiles.

In [ ]:
# Creamos 20 puntos percentiles igualmente distribuidos entre 0 y 100.
percs = numpy.linspace(0,100,21)

qn_sample = numpy.percentile(dataset_pf_identity_latin_america, percs)
qn_norm_dist = numpy.percentile(norm_dist.rvs(len(dataset_pf_identity_latin_america)), percs)

plt.figure(figsize=(12,6))
seaborn.regplot(x=qn_sample, y=qn_norm_dist)

plt.xlabel('Percentiles de la muestra de pf_identity_latin_america')
plt.ylabel('Percentiles de la distribución \n normal estimada')
plt.title('Gráfico QQ de la distribución de pf_identity_latin_america y una distribución normal')

x = numpy.linspace(numpy.min((qn_sample.min(), qn_norm_dist.min())), numpy.max((qn_sample.max(),qn_norm_dist.max())))
plt.plot(x,x, color="#FF5964", ls="--")

seaborn.despine()

### 3.3.3 Gráfico QQ de las distribuciones de hf_score en América Latina

Primero sacamos los parametros de la media y desviación estándar, para luego hacer una muestra aleatoria que si tenga una distribución normal que vamos a usar como comparador en el gráfico QQ

In [ ]:
from scipy import stats

# Definimos la variable para el dataset que vamos a utilizar
dataset_hf_score_latin_america = dataset[dataset['region'] == 'Latin America & the Caribbean']['hf_score'].dropna()

# Estimamos los parámetros muestrales (la media y desviación estándar)
loc, scale = stats.norm.fit(dataset_hf_score_latin_america)

# Creamos una distribución normal
norm_dist = stats.norm(loc, scale)

# Generamos una muestra
sample = norm_dist.rvs(1000)

# La graficamos
seaborn.distplot(sample, label='Distribución normal')
seaborn.distplot(dataset_hf_score_latin_america, label='hf_score_latin_america')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
seaborn.despine()

Ahora podemos armar el gráfico QQ. Usamos dos muestras del mismo tamaño, y calculamos los cuartiles.

In [ ]:
# Creamos 20 puntos percentiles igualmente distribuidos entre 0 y 100.
percs = numpy.linspace(0,100,21)

qn_sample = numpy.percentile(dataset_hf_score_latin_america, percs)
qn_norm_dist = numpy.percentile(norm_dist.rvs(len(dataset_hf_score_latin_america)), percs)

plt.figure(figsize=(12,6))
seaborn.regplot(x=qn_sample, y=qn_norm_dist)

plt.xlabel('Percentiles de la muestra de hf_score_america_latina')
plt.ylabel('Percentiles de la distribución \n normal estimada')
plt.title('Gráfico QQ de la distribución de hf_score_latin_america y una distribución normal')

x = numpy.linspace(numpy.min((qn_sample.min(), qn_norm_dist.min())), numpy.max((qn_sample.max(),qn_norm_dist.max())))
plt.plot(x,x, color="#FF5964", ls="--")

seaborn.despine()

### 3.3.4 Gráfico QQ de las distribuciones de hf_score en el mundo

Primero sacamos los parametros de la media y desviación estándar, para luego hacer una muestra aleatoria que si tenga una distribución normal que vamos a usar como comparador en el gráfico QQ

In [ ]:
from scipy import stats

# Definimos la variable para el dataset que vamos a utilizar
dataset_hf_score_mundo = dataset['hf_score'].dropna()

# Estimamos los parámetros muestrales (la media y desviación estándar)
loc, scale = stats.norm.fit(dataset_hf_score_mundo)

# Creamos una distribución normal
norm_dist = stats.norm(loc, scale)

# Generamos una muestra
sample = norm_dist.rvs(1000)

# La graficamos
seaborn.distplot(sample, label='Distribución normal')
seaborn.distplot(dataset_hf_score_mundo, label='hf_score_mundo')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
seaborn.despine()

Ahora podemos armar el gráfico QQ. Usamos dos muestras del mismo tamaño, y calculamos los cuartiles.

In [ ]:
# Creamos 20 puntos percentiles igualmente distribuidos entre 0 y 100.
percs = numpy.linspace(0,100,21)

qn_sample = numpy.percentile(dataset_hf_score_mundo, percs)
qn_norm_dist = numpy.percentile(norm_dist.rvs(len(dataset_hf_score_mundo)), percs)

plt.figure(figsize=(12,6))
seaborn.regplot(x=qn_sample, y=qn_norm_dist)

plt.xlabel('Percentiles de la muestra de hf_score_mundo')
plt.ylabel('Percentiles de la distribución \n normal estimada')
plt.title('Gráfico QQ de la distribución de hf_score_mundo y una distribución normal')

x = numpy.linspace(numpy.min((qn_sample.min(), qn_norm_dist.min())), numpy.max((qn_sample.max(),qn_norm_dist.max())))
plt.plot(x,x, color="#FF5964", ls="--")

seaborn.despine()

### 3.4.1 Skew/Curtosis pf_identity en América Latina y el mundo

In [ ]:
# Variables
dataset_pf_identity_latin_america = dataset[dataset['region'] == 'Latin America & the Caribbean']['pf_identity'].dropna()
dataset_pf_identity_mundo = dataset['pf_identity'].dropna()

# Skew
skew_pf_identity_latin_america = stats.skew (dataset_pf_identity_latin_america)
skew_pf_identity_mundo = stats.skew (dataset_pf_identity_mundo)

# Curtosis
kurtosis_pf_identity_latin_america = stats.kurtosis(dataset_pf_identity_latin_america)
kurtosis_pf_identity_mundo = stats.kurtosis(dataset_pf_identity_mundo)

# Impresión de resultados
print ('pf_identity en América Latina' , "\n")
print ('Skew : ' , skew_pf_identity_latin_america)
print ('Curtosis : ' , kurtosis_pf_identity_latin_america)
print ('---------------' + "\n")

print ('pf_identity en el mundo' , "\n")
print ('Skew : ' , skew_pf_identity_mundo)
print ('Curtosis: ' , kurtosis_pf_identity_mundo)

#### Analisis

Para pf_identity tanto en America Latina como en el mundo, indican

* skew negativo : la cola de distribución es asimetrica hacia la izquiera
* curtosis negativa  : hay una menor concentración de datos en torno a la media, excepto en America Latina que tiende a 0

### 3.4.2 Skew hf_score en América Latina y el mundo

In [ ]:
# Variables
dataset_hf_score_latin_america = dataset[dataset['region'] == 'Latin America & the Caribbean']['hf_score'].dropna()
dataset_hf_score_mundo = dataset['hf_score'].dropna()

# Skew
skew_hf_score_latin_america = stats.skew (dataset_hf_score_latin_america)
skew_hf_score_mundo = stats.skew (dataset_hf_score_mundo)

# Curtosis
kurtosis_hf_score_latin_america = stats.kurtosis(dataset_hf_score_latin_america)
kurtosis_hf_score_mundo = stats.kurtosis(dataset_hf_score_mundo)

# Impresión de resultados
print ('hf_score en América Latina' , "\n")
print ('Skew : ' , skew_hf_score_latin_america)
print ('Curtosis : ' , kurtosis_hf_score_latin_america)
print ('---------------' + "\n")

print ('hf_score en el mundo' , "\n")
print ('Skew : ' , skew_hf_score_mundo)
print ('Curtosis: ' , kurtosis_hf_score_mundo)

#### Analisis

Para hf_score tanto en America Latina como en el mundo, indican

* skew negativo : la cola de distribución es asimetrica hacia la izquiera
* curtosis  : Un valor de 5.3 nos indica una mayor concentración en el medio

## 4. Correlaciones

En este ejercicio queremos responder a las preguntas

* Las libertades sociales y económicas, ¿van siempre de la mano?
* ¿Cómo se relacionan ambas con las libertades individuales y respectivas a las relaciones personales?

Para ello, analizaremos las correlaciones entre las variables pf_identity, hf_score y ef_score. 

Como pf_indentity contribuye al cálculo de hf_score y ef_score, esperamos hallar algún grado de correlación. Sin embargo, queremos medir qué tanta correlación. 

1. ¿Qué conclusiones puede sacar de un gráfico pairplot de estas tres variables? ¿Es adecuado para los valores de pf_identity? ¿Por qué?
2. Graficar la correlación entre pf_identity y hf_score; y entre pf_identity y ef_score. Analizar el resultado, ¿se pueden sacar conclusiones? Tengan en cuenta que como pf_identity es el resultado de un promedio, sólo toma algunos valores. Es, en efecto, discreta.
3. Calcular algún coeficiente de correlación adecuado entre los dos pares de variables, dependiendo de la cantidad de datos, el tipo de datos y la distribución de los mismo. Algunas opciones son: coeficiente de pearson, coeficiente de spearman, coeficientes de tau y de kendall. Interpretar los resultados y justificar si las variables están correlacionadas o no. 
4. [Opcional] Analizar la correlación entre la region y el hf_score (y/o el ef_score); y entre la region y el pf_identity. Considerar que como la variable *region* es ordinal, debe utilizarse algún tipo de test. Explicar cuáles son los requisitos necesarios para la aplicación de ese test. (Si no se cumplieran, se pueden agregar algunos datos para generar más registros)


### 4.1 Graficar con pairplot las siguientes variables : pf_identity, hf_score y ef_score

In [ ]:
# Declaramos las variables a usar
dataset_latin_america = dataset[dataset['region'] == 'Latin America & the Caribbean']

plt.figure(figsize=(30,20))
seaborn.pairplot(dataset_latin_america, vars=['pf_identity', 'hf_score', 'ef_score'], markers='+')
seaborn.despine()

### 4.2.1 Graficar la correlación entre pf_identity y hf_score

In [ ]:
plt.figure(figsize=(15,10))
seaborn.scatterplot(data=dataset, x='pf_identity', y='hf_score')
seaborn.despine()

### 4.2.2 Graficar la correlación entre pf_identity y ef_score

In [ ]:
plt.figure(figsize=(15,10))
seaborn.scatterplot(data=dataset, x='pf_identity', y='ef_score')
seaborn.despine()

### 4.3 Calcular algún coeficiente de correlación adecuado entre los dos pares de variables

Dependiendo de la cantidad de datos, el tipo de datos y la distribución de los mismo. 

Algunas opciones son: 

* coeficiente de pearson
* coeficiente de spearman
* coeficientes de tau
* coeficientes de kendall

Interpretar los resultados y justificar si las variables están correlacionadas o no

In [ ]:
# Calculo de covarianza 

covarianza_pf_identity_hf_score = numpy.cov(dataset['pf_identity'].dropna(), dataset['hf_score'].dropna())[0][1]
covarianza_pf_identity_ef_score = numpy.cov(dataset['pf_identity'].dropna(), dataset['ef_score'].dropna())[0][1]
coeficiente_pf_identity_hf_score = dataset.corr(method="pearson")["pf_identity"]["hf_score"]
coeficiente_pf_identity_ef_score = dataset.corr(method="pearson")["pf_identity"]["ef_score"]

# Impresión de resultados

print ('\n'+'Covarianza'+'\n')
print ('Covarianza : pf_identity - hf_score: ', covarianza_pf_identity_hf_score )
print ('Covarianza : pf_identity - ef_score: ', covarianza_pf_identity_ef_score )
print ('\n'+'Coeficiente'+'\n')
print ('Coeficiente de Pearson : pf_identity - hf_score: ', coeficiente_pf_identity_hf_score )
print ('Coeficiente de Pearson : pf_identity - ef_score: ', coeficiente_pf_identity_ef_score )

#### Interpretación

##### Covarianzas

Dadas covarianzas positivas podemos decir que existe cierta relación entre los dos pares de variables. Siendo el par pf_identity - hf_score quien más fuerte relación parece mostrar

##### Coeficiente

Para ambos pares los coeficientes son mayores que cero, por lo tanto existe una correlación positiva